<a href="https://colab.research.google.com/github/SharpFFu/CC5349/blob/main/Comp5349_a2_500215470_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction
This notebook demonstrates a few useful methods for loading json file and for handling nested json objects. The example file is `test.json` in assignment 2. 

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 40.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=054a8ac59701a888b769b292fc309e6657fca53fd5542f4ff50e78eea3d07e52
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


### Connect to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Creat spark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("COMP5349 A2 500215470") \
    .config("spark.driver.memory", "16g")\
    .config("spark.sql.inMemoryColumnarStorage.compressed", "true")\
    .config("spark.sql.execution.arrow.enabled", "true")\
    .getOrCreate() 

### Load Json file as data frame

In [4]:
test_data = "/content/drive/MyDrive/comp5349/a2_data/test.json"
test_init_df = spark.read.json(test_data)

In [5]:
# The original file will be loaded into a data frame with one row and two columns
test_init_df.show(1)

+--------------------+--------+
|                data| version|
+--------------------+--------+
|[{[{Exhibit 10.16...|aok_v1.0|
+--------------------+--------+



In [6]:
test_init_df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- paragraphs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- context: string (nullable = true)
 |    |    |    |    |-- qas: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- answers: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |    |-- is_impossible: boolean (nullable = true)
 |    |    |    |    |    |    |-- question: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- version: string (nullable = true)



In [7]:
from pyspark.sql.functions import explode
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window, Row
from pyspark.sql.types import IntegerType, StringType, FloatType
import random

In [8]:
Test_data_df = test_init_df.select(explode("data").alias("data"))

1. paragraph

In [9]:
Test_data_df = Test_data_df.select(explode("data.paragraphs").alias("paragraph"))
Test_data_df.show(5)

+--------------------+
|           paragraph|
+--------------------+
|{Exhibit 10.16 SU...|
|{1               ...|
|{Exhibit 10.2\n\n...|
|{Exhibit 10.13\n\...|
|{ENDORSEMENT\n\nC...|
+--------------------+
only showing top 5 rows



2.paragraph+qas

In [10]:
Test_data_df = Test_data_df.select(col("paragraph.context").alias("paragraph_context"),explode("paragraph.qas").alias('qas'))
Test_data_df.show(5)


+--------------------+--------------------+
|   paragraph_context|                 qas|
+--------------------+--------------------+
|Exhibit 10.16 SUP...|{[{14, SUPPLY CON...|
|Exhibit 10.16 SUP...|{[{143, The selle...|
|Exhibit 10.16 SUP...|{[], LohaCompanyl...|
|Exhibit 10.16 SUP...|{[], LohaCompanyl...|
|Exhibit 10.16 SUP...|{[{10985, The Con...|
+--------------------+--------------------+
only showing top 5 rows



3.paragraph+(qas->)qas_question,qas_is_impossible,answer

In [11]:
Test_data_df =Test_data_df.select(col("paragraph_context"),
col("qas.question").alias("qas_question"),
col("qas.is_impossible").alias("qas_is_impossible"),explode_outer("qas.answers").alias('answers'),
)
Test_data_df.show(5)

+--------------------+--------------------+-----------------+--------------------+
|   paragraph_context|        qas_question|qas_is_impossible|             answers|
+--------------------+--------------------+-----------------+--------------------+
|Exhibit 10.16 SUP...|Highlight the par...|            false|{14, SUPPLY CONTR...|
|Exhibit 10.16 SUP...|Highlight the par...|            false|  {143, The seller:}|
|Exhibit 10.16 SUP...|Highlight the par...|            false|{49, The buyer/En...|
|Exhibit 10.16 SUP...|Highlight the par...|             true|                null|
|Exhibit 10.16 SUP...|Highlight the par...|             true|                null|
+--------------------+--------------------+-----------------+--------------------+
only showing top 5 rows



4.Paragraph+qas_question+qas_is_impossible+(answer)->answer_start,answer_text

In [12]:
Test_data_df = Test_data_df.select(col("paragraph_context"),
  col("qas_question"),
  col("qas_is_impossible"),
  col("answers.answer_start").alias("answer_start"),
  col("answers.text").alias("answer_text"),
)
Test_data_df.show(5)

+--------------------+--------------------+-----------------+------------+--------------------+
|   paragraph_context|        qas_question|qas_is_impossible|answer_start|         answer_text|
+--------------------+--------------------+-----------------+------------+--------------------+
|Exhibit 10.16 SUP...|Highlight the par...|            false|          14|     SUPPLY CONTRACT|
|Exhibit 10.16 SUP...|Highlight the par...|            false|         143|         The seller:|
|Exhibit 10.16 SUP...|Highlight the par...|            false|          49|The buyer/End-Use...|
|Exhibit 10.16 SUP...|Highlight the par...|             true|        null|                null|
|Exhibit 10.16 SUP...|Highlight the par...|             true|        null|                null|
+--------------------+--------------------+-----------------+------------+--------------------+
only showing top 5 rows



5.select qas_is_impossible == False

In [13]:
Test_data_P_df = Test_data_df.where(col("qas_is_impossible") == False)

In [14]:
Test_data_P_df.show(5)

+--------------------+--------------------+-----------------+------------+--------------------+
|   paragraph_context|        qas_question|qas_is_impossible|answer_start|         answer_text|
+--------------------+--------------------+-----------------+------------+--------------------+
|Exhibit 10.16 SUP...|Highlight the par...|            false|          14|     SUPPLY CONTRACT|
|Exhibit 10.16 SUP...|Highlight the par...|            false|         143|         The seller:|
|Exhibit 10.16 SUP...|Highlight the par...|            false|          49|The buyer/End-Use...|
|Exhibit 10.16 SUP...|Highlight the par...|            false|       10985|The Contract is v...|
|Exhibit 10.16 SUP...|Highlight the par...|            false|       10691|It will be govern...|
+--------------------+--------------------+-----------------+------------+--------------------+
only showing top 5 rows



Count the number of contracts that contain poss_record for each question

In [15]:
pos_contract_N = Test_data_P_df.groupby("paragraph_context").count().withColumnRenamed("count","positive_contract_num")
pos_contract_N.show(5)

+--------------------+---------------------+
|   paragraph_context|positive_contract_num|
+--------------------+---------------------+
|Exhibit 10.1\n\n[...|                   43|
|EXHIBIT 10.25    ...|                   22|
|EXHIBIT 99.4\n\n ...|                   26|
|Exhibit 10.13\n\n...|                   83|
|Exhibit 10.16 SUP...|                    7|
+--------------------+---------------------+
only showing top 5 rows



Convert to RDD

In [172]:
Test_RDD = Test_data_P_df.rdd.map(lambda x:(x[0], x[1], x[2], x[3], x[4])).cache()

Cut data to the positive samples and Possible negative samples( stride = 2048 window = 4096)

In [173]:
def sample_pos(line):
    stride = 2048
    window = 4096
    result = []
    length = len(line[0])
    loop_num = int(length / stride) + 1
    index = 0
    neg_result = []
    for i in range(loop_num):
        c_start = line[3]
        c_end = line[3] + len(line[4])
        if (i *stride>c_end) or (i*stride+window <c_start):
          neg_result.append(Row(source=line[0][i * stride: i * stride + window], qas_question=line[1], answer_start=0, answer_end=0, type_name="possible negative"))  
        elif(i* stride <= c_start < i * stride + window) :
          if i * stride + window > c_end:
           index_end=c_end-i*stride 
           if index_end>=window:
             index_end=window
           result.append(Row(source=line[0][i * stride: i * stride + window], qas_question=line[1], answer_start=line[3] % stride, answer_end=index_end, type_name="positive"))
           index += 1
          else :
           index_end=window
           result.append(Row(source=line[0][i * stride: i * stride + window], qas_question=line[1], answer_start=line[3] % stride, answer_end=index_end, type_name="positive"))
           index += 1
        elif ( i * stride + window>i* stride >= c_start):
          if i * stride + window > c_end:
           index_end=i*stride-c_end
           if index_end>=window:
             index_end=window
           result.append(Row(source=line[0][i * stride: i * stride + window], qas_question=line[1], answer_start=0, answer_end=index_end, type_name="positive"))
           index += 1
          else :
           result.append(Row(source=line[0][i * stride: i * stride + window], qas_question=line[1], answer_start=0, answer_end=window, type_name="positive"))  
           index += 1          
    result.extend(neg_result[:index])
    return result

In [174]:

pos_Sample_rdd= Test_RDD.flatMap(sample_pos)
pos_Sample_df= spark.createDataFrame(pos_Sample_rdd).cache()

In [175]:
pos_Sample_df.show(5)

+--------------------+--------------------+------------+----------+-----------------+
|              source|        qas_question|answer_start|answer_end|        type_name|
+--------------------+--------------------+------------+----------+-----------------+
|Exhibit 10.16 SUP...|Highlight the par...|          14|        29|         positive|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|possible negative|
|Exhibit 10.16 SUP...|Highlight the par...|         143|       154|         positive|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|possible negative|
|Exhibit 10.16 SUP...|Highlight the par...|          49|       117|         positive|
+--------------------+--------------------+------------+----------+-----------------+
only showing top 5 rows



In [177]:
pos_result = pos_Sample_df.select("source","qas_question","answer_start","answer_end")
pos_count = pos_Sample_df.where(col("type_name")=="positive").groupBy("qas_question").count()

In [178]:
pos_result.show(5)

+--------------------+--------------------+------------+----------+
|              source|        qas_question|answer_start|answer_end|
+--------------------+--------------------+------------+----------+
|Exhibit 10.16 SUP...|Highlight the par...|          14|        29|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|
|Exhibit 10.16 SUP...|Highlight the par...|         143|       154|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|
|Exhibit 10.16 SUP...|Highlight the par...|          49|       117|
+--------------------+--------------------+------------+----------+
only showing top 5 rows



In [179]:
pos_count.show(5)

+--------------------+-----+
|        qas_question|count|
+--------------------+-----+
|Highlight the par...|  257|
|Highlight the par...|  289|
|Highlight the par...|  141|
|Highlight the par...|  224|
|Highlight the par...|  254|
+--------------------+-----+
only showing top 5 rows



In [180]:
Test_data_IM_df = Test_data_df.where(col("qas_is_impossible") == True).join(pos_count,"qas_question")

In [181]:
Test_data_IM_df.show(5)

+--------------------+--------------------+-----------------+------------+-----------+-----+
|        qas_question|   paragraph_context|qas_is_impossible|answer_start|answer_text|count|
+--------------------+--------------------+-----------------+------------+-----------+-----+
|Highlight the par...|EXHIBIT 99.4\n\n ...|             true|        null|       null|  257|
|Highlight the par...|Exhibit 10.1 JOIN...|             true|        null|       null|  257|
|Highlight the par...|Exhibit 10.5   IN...|             true|        null|       null|  257|
|Highlight the par...|Exhibit 10.26\n\n...|             true|        null|       null|  257|
|Highlight the par...|Exhibit 4.5\n\nCO...|             true|        null|       null|  257|
+--------------------+--------------------+-----------------+------------+-----------+-----+
only showing top 5 rows



In [198]:
impos_sample_df = Test_data_IM_df.join(pos_contract_N ,"paragraph_context").cache()

In [199]:
impos_sample_df.show(5)

+--------------------+--------------------+-----------------+------------+-----------+-----+---------------------+
|   paragraph_context|        qas_question|qas_is_impossible|answer_start|answer_text|count|positive_contract_num|
+--------------------+--------------------+-----------------+------------+-----------+-----+---------------------+
|EXHIBIT 99.4\n\n ...|Highlight the par...|             true|        null|       null|  257|                   26|
|Exhibit 10.1 JOIN...|Highlight the par...|             true|        null|       null|  257|                   19|
|Exhibit 10.5   IN...|Highlight the par...|             true|        null|       null|  257|                   41|
|Exhibit 10.26\n\n...|Highlight the par...|             true|        null|       null|  257|                   11|
|Exhibit 4.5\n\nCO...|Highlight the par...|             true|        null|       null|  257|                   13|
+--------------------+--------------------+-----------------+------------+------

In [184]:
def sample_impos_count(count, positive_contract_count):
  caclunumber= float(count / positive_contract_count)
  result = int(count / positive_contract_count)
  if caclunumber<result:
    result+=1
  return result

In [200]:
udf_fc = udf(lambda x,y:sample_impos_count(x,y))

In [201]:
impos_sample_df = impos_sample_df.withColumn("impossible_count",udf_fc(col("count"),col("positive_contract_num"))).select("*",round("impossible_count")).withColumnRenamed("round(impossible_count, 0)","impossible_count_result")

In [202]:
impos_sample_df  = impos_sample_df .select("paragraph_context","qas_question","qas_is_impossible","answer_start","answer_text","impossible_count_result")

In [203]:
impos_sample_df .show(5)

+--------------------+--------------------+-----------------+------------+-----------+-----------------------+
|   paragraph_context|        qas_question|qas_is_impossible|answer_start|answer_text|impossible_count_result|
+--------------------+--------------------+-----------------+------------+-----------+-----------------------+
|EXHIBIT 99.4\n\n ...|Highlight the par...|             true|        null|       null|                    9.0|
|Exhibit 10.1 JOIN...|Highlight the par...|             true|        null|       null|                   13.0|
|Exhibit 10.5   IN...|Highlight the par...|             true|        null|       null|                    6.0|
|Exhibit 10.26\n\n...|Highlight the par...|             true|        null|       null|                   23.0|
|Exhibit 4.5\n\nCO...|Highlight the par...|             true|        null|       null|                   19.0|
+--------------------+--------------------+-----------------+------------+-----------+-----------------------+
o

In [204]:
impos_sample_rdd = impos_sample_df.rdd.map(lambda x:(x[0], x[1], x[2], x[3], x[4], x[5]))

In [205]:
def sample_impos(line):
  stride = 2048
  window_size = 4096
  result = []
  source_number = int(line[5])
  context_length = len(line[1])
  if(context_length % stride ==0):
    times = context_length / stride
  else:
    times = int(context_length / stride) + 1
  for i in range(times):
      result.append(Row(source=line[1][i * stride: i * stride + window_size], question=line[0], answer_start=0, answer_end=0))
  return result[:source_number]


In [206]:
Ne_rdd = impos_sample_rdd.flatMap(sample_impos)

In [207]:
Ne_result = spark.createDataFrame(Ne_rdd).cache()
F_result = pos_result.union(Ne_result)

In [210]:
F_result = F_result.withColumnRenamed("qas_question","question")


In [211]:
F_result.show()

+--------------------+--------------------+------------+----------+
|              source|            question|answer_start|answer_end|
+--------------------+--------------------+------------+----------+
|Exhibit 10.16 SUP...|Highlight the par...|          14|        29|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|
|Exhibit 10.16 SUP...|Highlight the par...|         143|       154|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|
|Exhibit 10.16 SUP...|Highlight the par...|          49|       117|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|
|e Sellers shall a...|Highlight the par...|         745|      2857|
|e decision of the...|Highlight the par...|         745|       809|
|Exhibit 10.16 SUP...|Highlight the par...|           0|         0|
| F-1, 12/9/2019\n...|Highlight the par...|           0|         0|
|e Sellers shall a...|Highlight the par...|         451|      2667|
|e decision of the...|Highlight the par...|     

In [158]:
result.count()

12101

In [160]:
result.write.json("output_test.json")